In [11]:
import cv2
from imwatermark import WatermarkEncoder
import os
from blind_watermark import WaterMark

# Part A: Encode

In [12]:
#Encode DWT A

infolder='in_img'
outfolder=os.path.join('out_img','algo_a')
in_img_list = os.listdir(infolder)
text='zhen'

for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_A_'+img
    out_path = os.path.join(outfolder,out_name)
    
   
    encoder = WaterMark(password_img=1, password_wm=1)
    encoder.read_img(in_path)
    encoder.read_wm(text, mode='str')
    encoder.embed(out_path)
     
    print("Done Encoding: " + out_path)

Done Encoding: out_img\algo_a\Algo_A_MRI_A.png
Done Encoding: out_img\algo_a\Algo_A_MRI_B.png
Done Encoding: out_img\algo_a\Algo_A_MRI_C.png
Done Encoding: out_img\algo_a\Algo_A_MRI_D.png
Done Encoding: out_img\algo_a\Algo_A_MRI_E.png
Done Encoding: out_img\algo_a\Algo_A_MRI_F.png
Done Encoding: out_img\algo_a\Algo_A_MRI_G.png
Done Encoding: out_img\algo_a\Algo_A_MRI_H.png
Done Encoding: out_img\algo_a\Algo_A_MRI_I.png
Done Encoding: out_img\algo_a\Algo_A_MRI_J.png
Done Encoding: out_img\algo_a\Algo_A_MRI_K.png
Done Encoding: out_img\algo_a\Algo_A_MRI_L.png
Done Encoding: out_img\algo_a\Algo_A_MRI_M.png
Done Encoding: out_img\algo_a\Algo_A_MRI_N.png
Done Encoding: out_img\algo_a\Algo_A_MRI_O.png
Done Encoding: out_img\algo_a\Algo_A_MRI_P.png
Done Encoding: out_img\algo_a\Algo_A_MRI_Q.png
Done Encoding: out_img\algo_a\Algo_A_MRI_R.png
Done Encoding: out_img\algo_a\Algo_A_MRI_S.png
Done Encoding: out_img\algo_a\Algo_A_MRI_T.png


In [13]:
#Encode dwtDct B

infolder='in_img'
outfolder=os.path.join('out_img','algo_b')
in_img_list = os.listdir(infolder)
text='zhen'

encoder = WatermarkEncoder()


for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_B_'+img
    out_path = os.path.join(outfolder,out_name)
    
    in_img = cv2.imread(in_path)
    
    encoder.set_watermark('bytes', text.encode('utf-8'))
    
    img_encoded = encoder.encode(in_img,'dwtDct')
    
    cv2.imwrite(out_path,img_encoded)
    
    print("Done Encoding: " + out_path)
    


Done Encoding: out_img\algo_b\Algo_B_MRI_A.png
Done Encoding: out_img\algo_b\Algo_B_MRI_B.png
Done Encoding: out_img\algo_b\Algo_B_MRI_C.png
Done Encoding: out_img\algo_b\Algo_B_MRI_D.png
Done Encoding: out_img\algo_b\Algo_B_MRI_E.png
Done Encoding: out_img\algo_b\Algo_B_MRI_F.png
Done Encoding: out_img\algo_b\Algo_B_MRI_G.png
Done Encoding: out_img\algo_b\Algo_B_MRI_H.png
Done Encoding: out_img\algo_b\Algo_B_MRI_I.png
Done Encoding: out_img\algo_b\Algo_B_MRI_J.png
Done Encoding: out_img\algo_b\Algo_B_MRI_K.png
Done Encoding: out_img\algo_b\Algo_B_MRI_L.png
Done Encoding: out_img\algo_b\Algo_B_MRI_M.png
Done Encoding: out_img\algo_b\Algo_B_MRI_N.png
Done Encoding: out_img\algo_b\Algo_B_MRI_O.png
Done Encoding: out_img\algo_b\Algo_B_MRI_P.png
Done Encoding: out_img\algo_b\Algo_B_MRI_Q.png
Done Encoding: out_img\algo_b\Algo_B_MRI_R.png
Done Encoding: out_img\algo_b\Algo_B_MRI_S.png
Done Encoding: out_img\algo_b\Algo_B_MRI_T.png


In [15]:
#encode DwtDctSvd C

infolder='in_img'
outfolder=os.path.join('out_img','algo_c')
in_img_list = os.listdir(infolder)
text='zhen'

encoder = WatermarkEncoder()

for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_C_'+img
    out_path = os.path.join(outfolder,out_name)
    
    in_img = cv2.imread(in_path)
    
    encoder.set_watermark('bytes', text.encode('utf-8'))
    
    img_encoded = encoder.encode(in_img,'dwtDctSvd')
    
    cv2.imwrite(out_path,img_encoded)
    
    print("Done Encoding: " + out_path)
    


Done Encoding: out_img\algo_c\Algo_C_MRI_A.png
Done Encoding: out_img\algo_c\Algo_C_MRI_B.png
Done Encoding: out_img\algo_c\Algo_C_MRI_C.png
Done Encoding: out_img\algo_c\Algo_C_MRI_D.png
Done Encoding: out_img\algo_c\Algo_C_MRI_E.png
Done Encoding: out_img\algo_c\Algo_C_MRI_F.png
Done Encoding: out_img\algo_c\Algo_C_MRI_G.png
Done Encoding: out_img\algo_c\Algo_C_MRI_H.png
Done Encoding: out_img\algo_c\Algo_C_MRI_I.png
Done Encoding: out_img\algo_c\Algo_C_MRI_J.png
Done Encoding: out_img\algo_c\Algo_C_MRI_K.png
Done Encoding: out_img\algo_c\Algo_C_MRI_L.png
Done Encoding: out_img\algo_c\Algo_C_MRI_M.png
Done Encoding: out_img\algo_c\Algo_C_MRI_N.png
Done Encoding: out_img\algo_c\Algo_C_MRI_O.png
Done Encoding: out_img\algo_c\Algo_C_MRI_P.png
Done Encoding: out_img\algo_c\Algo_C_MRI_Q.png
Done Encoding: out_img\algo_c\Algo_C_MRI_R.png
Done Encoding: out_img\algo_c\Algo_C_MRI_S.png
Done Encoding: out_img\algo_c\Algo_C_MRI_T.png


In [16]:
#encode RivaGAN D

#encode DwtDctSvd C

infolder='in_img'
outfolder=os.path.join('out_img','algo_d')
in_img_list = os.listdir(infolder)
text='zhen'

encoder = WatermarkEncoder()

for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_D_'+img
    out_path = os.path.join(outfolder,out_name)
    
    in_img = cv2.imread(in_path)
    
    encoder.set_watermark('bytes', text.encode('utf-8'))
    encoder.loadModel()
    img_encoded = encoder.encode(in_img,'rivaGan')
    
    cv2.imwrite(out_path,img_encoded)
    
    print("Done Encoding: " + out_path)
    


Done Encoding: out_img\algo_d\Algo_D_MRI_A.png
Done Encoding: out_img\algo_d\Algo_D_MRI_B.png
Done Encoding: out_img\algo_d\Algo_D_MRI_C.png
Done Encoding: out_img\algo_d\Algo_D_MRI_D.png
Done Encoding: out_img\algo_d\Algo_D_MRI_E.png
Done Encoding: out_img\algo_d\Algo_D_MRI_F.png
Done Encoding: out_img\algo_d\Algo_D_MRI_G.png
Done Encoding: out_img\algo_d\Algo_D_MRI_H.png
Done Encoding: out_img\algo_d\Algo_D_MRI_I.png
Done Encoding: out_img\algo_d\Algo_D_MRI_J.png
Done Encoding: out_img\algo_d\Algo_D_MRI_K.png
Done Encoding: out_img\algo_d\Algo_D_MRI_L.png
Done Encoding: out_img\algo_d\Algo_D_MRI_M.png
Done Encoding: out_img\algo_d\Algo_D_MRI_N.png
Done Encoding: out_img\algo_d\Algo_D_MRI_O.png
Done Encoding: out_img\algo_d\Algo_D_MRI_P.png
Done Encoding: out_img\algo_d\Algo_D_MRI_Q.png
Done Encoding: out_img\algo_d\Algo_D_MRI_R.png
Done Encoding: out_img\algo_d\Algo_D_MRI_S.png
Done Encoding: out_img\algo_d\Algo_D_MRI_T.png


# Part B: Attacking

# Part C: Decode